In [ ]:
# Libraries to be used for the model (project)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from lifelines import WeibullFitter
from lifelines import KaplanMeierFitter
from lifelines import ExponentialFitter
from lifelines import LogNormalFitter
from lifelines.plotting import plot_lifetimes

# dataset containing 299 rows and 13 columns was used for this analysis
dat=pd.read_csv("Desktop/Data.csv") 
display(dat)

# The next step is to clean the data, but first we examined data structure of each variables type
# By simple observation, we can tell that some columns are boolen or binary values. Meaning they can only assume 2 values
# either 1 or 2, true/false, yes/no, male/female

# we extracted those columns and ran a test on the reminder columns to check for missing values

dat2=dat[['age','creatinine_phosphokinase','ejection_fraction',
             'platelets','serum_creatinine','serum_sodium','time']]

pd.isna(dat2) # pd.isna to check missing values

# There are no missing values which indicates the data is cleaned.

# Data description for continuous variable in the data
dat2.describe()

# for categorical variable in the data
categoricaldata=pd.read_csv("Desktop/catbook.csv") # dataset containing 299 rows and 13 columns was used for this analysis
display(categoricaldata)
# Table below shows the percentage summary of the different categorical variables in the data. 
# Its also further split into dead and survived patients.

# We also used Correlation matrix to determine the relationship between features that are highly correlated
round(dat.corr(),3)

import seaborn as sns
# Correlation visualization to show continuous features that are highly correlated
plt.figure(figsize=[20,10])
matrix=round(dat.corr(),3)
sns.heatmap(matrix,annot=True,cmap="YlGnBu")  # cmap="YlGnBu"
plt.show()

# lifelines plot of the dataset to show days survived and death event occurence
df=dat.sample(25)  # random sampling of data
plt.figure(figsize=[15,7])
plot_lifetimes(df['time'])
plt.title('lifetimes plot in days')
plt.xlabel('time in days')
plt.ylabel('patients')
plt.show()

# Survival model to predict patients death rate and survival probability
T=dat[dat['DEATH_EVENT']==1]['time']  # 1 means a death_event occured, while 0 is patient survived
#plt.figure(figsize=[15,7])           # we selected death event to visualize how long they survived
plt.hist(T,bins=20)
#plt.hist(T)

plt.title('Survival time in days')
plt.xlabel('time in days')
plt.ylabel('Freq of deaths')
plt.show()

# Next we calculate the cummulative failure probability, to show the chance of failure (death) by time (days)
dsort=dat.sort_values(by='time')
#display(dsort)
cumdat=dsort[dsort['DEATH_EVENT']==1] # exact failure data after removing censored data
#display(cumdat)
cumfailure=cumdat['DEATH_EVENT'].cumsum()    # cumulative failure data
#Ft=cumfailure/299   # failure rate
Ft=(cumfailure-0.3)/(299+0.4) # failure rate using rank
plt.plot(cumdat['time'],Ft,label='cummulative failure (deaths)')
plt.axvline(x=249, color='b')
# kaplanmeier
km=KaplanMeierFitter()
kmf=km.fit(dat['time'],dat['DEATH_EVENT'])
#kmf.plot()
kmf.plot_cumulative_density(label='kaplanMeier')
plt.legend()
plt.show()

# Now we compared different distribution models to to check which 
# model will accurately predict patients death rate and survival probability
# For this we used maximum likelihood estimation and AIC index to check model fitting

print('############################## exponential ###################################')
# exponential distribution
ef=ExponentialFitter().fit(dat['time'],dat['DEATH_EVENT'])
display(ef.summary)
print ("AIC of Exponential model is: ",ef.AIC_)  # AIC index- smallest AIC index is a better fit
exp=ef.log_likelihood_
print('The loglikelihood for exponential dist is', round(exp,2)) # highest loglihood is a better fit because we want maximize loglihood
#ef.print_summary()

print('\n############################## weibull ###################################')
# weibull
wf=WeibullFitter().fit(dat['time'],dat['DEATH_EVENT'])
display(wf.summary)
print ("AIC of Weibull model is: ", wf.AIC_) 
wei=wf.log_likelihood_
print('The loglikelihood for weibull dist is', round(wei,2))
#wf.print_summary()

# survival probability plot of different distribution models in comparism with kaplanmeier probability plot

# probability plots of different models
plt.figure(figsize=[16,4])
plt.subplot(1,2,1)  # # exponential
ef.plot_survival_function()
plt.xlabel('Days Survived')
plt.ylabel('Survival Prob')

plt.subplot(1,2,2)  # weibull
wf.plot_survival_function()
plt.xlabel('Days Survived')
plt.ylabel('Survival Prob')
plt.show()

#wf.plot_hazard()
#plt.xlabel('Days Survived')
#plt.ylabel('Hazard Rate')
#plt.show()


# Comparing survival curve of 2 models with kaplanMeier
print('############################## Comparing survival curve ###################################')

# kaplanMeier
plt.figure(figsize=[12,10])
km.plot(label='kaplanMeier')
#weibull distribution
wf.plot_survival_function(label='weibull')
# exponential distribution
ef.plot_survival_function(label='exponential')

plt.xlabel('Days Survived')
plt.ylabel('Survival Prob')
plt.show()

# probability data ploting is another way to check model fitting and also helps in validating maximum likelihood estimation

print('############################## exponential ###################################')

# exponential
# Assuming the life-time of the patients follow exponential distribution, we can fit the model in 
# ordinary least squares regression to estimate coefficients that helps us to visualise the data
# Input and output is derived by using exponential distribution formula for reliability -ln(1-Ft)=lambda*time

#cumdat
expl=cumdat['time']
lnft=-np.log(1-Ft)  
elm=sm.OLS(lnft,expl)  # because its exponential, it doesn't require a constant
res_elm=elm.fit()
print(res_elm.summary())
print('lambda is:', res_elm.params.time)

plt.figure(figsize=[12,9])
plt.subplot(1,2,1)
plt.scatter(expl,lnft,label='Obs')
plt.plot(expl,res_elm.predict(expl) ,label='Exp model')
plt.title('Exponential probability plot')
plt.xlabel('survival time in days')
plt.ylabel('-ln(1-Ft)')
plt.legend()
print('\n############################## weibull ###################################')

# weibull formula for reliability ln(-ln(1-Ft))= beta*ln*time - beta*ln*alpha
weil= np.log(cumdat['time'])  
xweil= sm.add_constant(weil) # weibull requires a constant because it has slope and intersect
yweil= np.log(lnft)
wlm=sm.OLS(yweil,xweil)
res_wlm=wlm.fit()
print(res_wlm.summary())
print('alpha is:', np.exp(-(res_wlm.params.const/res_wlm.params.time)))
print('beta is:', 1/res_wlm.params.time)

plt.subplot(1,2,2)
plt.scatter(weil,yweil,label='Obs')
plt.plot(weil,res_wlm.predict(xweil) ,label='Weibul model')
plt.title('Weibull probability plot')
plt.xlabel('survival time in days')
plt.ylabel('ln(-ln(1-Ft))')
plt.legend()
plt.show()

# age	creatinine_phosphokinase	ejection_fraction	platelets	serum_creatinine	serum_sodium	DEATH_EVENT
cumdat2=cumdat[['age','creatinine_phosphokinase','ejection_fraction',
                'platelets','serum_creatinine','serum_sodium']]    # subseting continuous features
cumdat2

# using weibull survival model to observe other features against patients death
for column in cumdat2:
    
    print('\n##############################',column,'###################################')
    wf=WeibullFitter().fit(cumdat2[column],cumdat['DEATH_EVENT'])
    display(wf.summary)
    
    wf.plot_survival_function(label='weibul')
    plt.xlabel(column)
    plt.ylabel('Survival Prob')
    plt.show()
    
# checking accuracy of model using CoxPHFitter from lifelines

from lifelines import CoxPHFitter
cph=CoxPHFitter()
cph.fit(dat, duration_col='time',event_col='DEATH_EVENT')
display(cph.summary)
display(cph.check_assumptions(dat, show_plots=True))

# kaplan Meier curve to compare the difference high risk continuous features

#wf=WeibullFitter().fit(cumdat2['serum_creatinine'],cumdat['DEATH_EVENT'])

plt.figure(figsize=[16,5])
plt.subplot(1,2,1)
km1=KaplanMeierFitter().fit(dat['serum_creatinine'],weights=dat['DEATH_EVENT'])
km1.plot(label='serum_creatinine')
plt.xlabel('serum_creatinine')
plt.ylabel('Survival Prob')
plt.title('kaplan meier model')

plt.subplot(1,2,2)
km2=KaplanMeierFitter().fit(dat['ejection_fraction'],weights=dat['DEATH_EVENT'])
km2.plot(label='ejection_fraction')
plt.xlabel('ejection_fraction')
plt.ylabel('Survival Prob')

plt.title('kaplan meier model')
plt.show()

# kaplan Meier curve to compare the difference between each categorical feature
catdata=cumdat[['anaemia','diabetes','high_blood_pressure','sex','smoking','time','DEATH_EVENT']]

#print('\n############################## anaemia ###################################')
pcatdata=catdata[catdata['anaemia']==1] # this code indicates that there is presence / 1 also indicates male for sex feature
npcatdata=catdata[catdata['anaemia']==2] # this code indicates that there is no presence / 2 also indicates female for sex feature

akm1=KaplanMeierFitter().fit(pcatdata['time'],weights=pcatdata['DEATH_EVENT'])
akm2=KaplanMeierFitter().fit(npcatdata['time'],weights=npcatdata['DEATH_EVENT'])

#print('\n############################## diabetes ###################################')
pcatdata=catdata[catdata['diabetes']==1] # this code indicates that there is presence / 1 also indicates male for sex feature
npcatdata=catdata[catdata['diabetes']==2] # this code indicates that there is no presence / 2 also indicates female for sex feature

dkm1=KaplanMeierFitter().fit(pcatdata['time'],weights=pcatdata['DEATH_EVENT'])
dkm2=KaplanMeierFitter().fit(npcatdata['time'],weights=npcatdata['DEATH_EVENT'])

#print('\n############################## high_blood_pressure ###################################')
pcatdata=catdata[catdata['high_blood_pressure']==1] # this code indicates that there is presence / 1 also indicates male for sex feature
npcatdata=catdata[catdata['high_blood_pressure']==2] # this code indicates that there is no presence / 2 also indicates female for sex feature

hkm1=KaplanMeierFitter().fit(pcatdata['time'],weights=pcatdata['DEATH_EVENT'])
hkm2=KaplanMeierFitter().fit(npcatdata['time'],weights=npcatdata['DEATH_EVENT'])

#print('\n############################## smoking ###################################')
pcatdata=catdata[catdata['smoking']==1] # this code indicates that there is presence / 1 also indicates male for sex feature
npcatdata=catdata[catdata['smoking']==2] # this code indicates that there is no presence / 2 also indicates female for sex feature

skm1=KaplanMeierFitter().fit(pcatdata['time'],weights=pcatdata['DEATH_EVENT'])
skm2=KaplanMeierFitter().fit(npcatdata['time'],weights=npcatdata['DEATH_EVENT'])

#print('\n############################## sex ###################################')
pcatdata=catdata[catdata['sex']==1] # 1 indicates male for sex feature
npcatdata=catdata[catdata['sex']==2] # 2 indicates female for sex feature

mkm1=KaplanMeierFitter().fit(pcatdata['time'],weights=pcatdata['DEATH_EVENT'])
fkm2=KaplanMeierFitter().fit(npcatdata['time'],weights=npcatdata['DEATH_EVENT'])

# categorical features with high risk associated to patients survival
# using 50 days as survival time to check survival probability rate for each feature

plt.figure(figsize=[18,6])
plt.subplot(1,3,1)
skm1.plot(label='presence')
skm2.plot(label='no presence')
plt.title('survival plot of smoking')
plt.xlabel('smoking')
plt.ylabel('Survival Prob')
plt.axvline(x=50, color='b')

plt.subplot(1,3,2)
hkm1.plot(label='presence')
hkm2.plot(label='no presence')
plt.title('survival plot of high_blood_pressure')
plt.xlabel('high_blood_pressure')
plt.ylabel('Survival Prob')
plt.axvline(x=50, color='b')

plt.subplot(1,3,3)
mkm1.plot(label='male')
fkm2.plot(label='female')
plt.title('survival plot of sex')
plt.xlabel('sex')
plt.ylabel('Survival Prob')

plt.axvline(x=50, color='b')
plt.show()

# Other low risk categorical features which are not associated with patient's survival

plt.figure(figsize=[20,7])
plt.subplot(1,3,1)
akm1.plot(label='presence')
akm2.plot(label='no presence')
plt.title('survival plot of anaemia')
plt.xlabel('anaemia')
plt.ylabel('Survival Prob')
plt.axvline(x=100, color='b')
plt.subplot(1,3,2)
dkm1.plot(label='presence')
dkm2.plot(label='no presence')
plt.title('survival plot of diabetes')
plt.xlabel('diabetes')
plt.ylabel('Survival Prob')
plt.axvline(x=100, color='b')